In [2]:
# Imports and import vars
import os
import re
dir = "./outputs/new_llava_trials/"


In [3]:
overall_times = []
overall_average_times = []

for file in os.listdir(dir):
    with open(f"{dir}/{file}", 'r') as f:
        content = f.read()
        #Convert to dictionary
        content = eval(content)
        
    times = []
    average_times = []
    #Some elements are lists. For those are are, grab the second element and store it in a list
    #This is for the case of the llava trials
    for key in content:
        if type(content[key]) == list:
            # For each element in the list, keep iterating until it finds the substring "Time completed:"
            for element in content[key]:
                if "Time completed:" in element:
                    #Now that we are here, only grab the numbers in the string
                    time_to_complete = element.split(" ")[2]
                    average_time = element.split(" ")[4][1:]
                    times.append(time_to_complete)
                    average_times.append(average_time)

    # Calculate a 5 number summary and means of the times and average times
    times = [float(i) for i in times]
    average_times = [float(i) for i in average_times]
    for i in range(len(times)):
        overall_times.append(times[i])
        overall_average_times.append(average_times[i])

    
    times.sort()
    average_times.sort()

    # Time to calculate the 5 number summary and mean
    print(f"File: {file}")
    try:
        min_time = times[0]
        max_time = times[-1]
        q1_time = times[int(len(times)/4)]
        q2_time = times[int(len(times)/2)]
        q3_time = times[int(3*len(times)/4)]
        mean_time = sum(times)/len(times)
        min_avg_time = average_times[0]
        max_avg_time = average_times[-1]
        q1_avg_time = average_times[int(len(average_times)/4)]
        q2_avg_time = average_times[int(len(average_times)/2)]
        q3_avg_time = average_times[int(3*len(average_times)/4)]
        mean_avg_time = sum(average_times)/len(average_times)
        print(f"Times: [{min_time}, {q1_time}, {q2_time}, {q3_time}, {max_time}] ({mean_time:.3f})")
        print(f"Average Times (seconds per image): [{min_avg_time}, {q1_avg_time}, {q2_avg_time}, {q3_avg_time}, {max_avg_time}] ({mean_avg_time:.3f})")
    except IndexError:
        print("No times to calculate")
        print("No average times to calculate")
    print("\n\n")

#Full 5 number summary and mean
overall_times.sort()
overall_average_times.sort()
print("Overall Times")
try:
    min_time = overall_times[0]
    max_time = overall_times[-1]
    q1_time = overall_times[int(len(overall_times)/4)]
    q2_time = overall_times[int(len(overall_times)/2)]
    q3_time = overall_times[int(3*len(overall_times)/4)]
    mean_time = sum(overall_times)/len(overall_times)
    min_avg_time = overall_average_times[0]
    max_avg_time = overall_average_times[-1]
    q1_avg_time = overall_average_times[int(len(overall_average_times)/4)]
    q2_avg_time = overall_average_times[int(len(overall_average_times)/2)]
    q3_avg_time = overall_average_times[int(3*len(overall_average_times)/4)]
    mean_avg_time = sum(overall_average_times)/len(overall_average_times)
    print(f"Times: [{min_time}, {q1_time}, {q2_time}, {q3_time}, {max_time}] ({mean_time:.3f})")
    print(f"Average Times (seconds per image): [{min_avg_time}, {q1_avg_time}, {q2_avg_time}, {q3_avg_time}, {max_avg_time}] ({mean_avg_time:.3f})")
except IndexError:
    print("No times to calculate")
    print("No average times to calculate")

File: accidentsandvehicles.txt
No times to calculate
No average times to calculate



File: accidenttypes.txt
No times to calculate
No average times to calculate



File: controlgroup.txt
Times: [4.590463, 31.502058, 40.182878, 76.193003, 85.216833] (46.433)
Average Times (seconds per image): [0.524, 3.568, 4.664, 8.173, 80.719] (8.822)



File: fulltestnewprompt.txt
Times: [23.954611, 36.248197, 46.0113, 56.3674, 96.197191] (48.211)
Average Times (seconds per image): [0.617, 3.031, 4.419, 7.279, 28.77] (6.201)



File: fulltestnewpromptshuffled.txt
Times: [25.056979, 38.385919, 47.715889, 56.491967, 112.321116] (51.987)
Average Times (seconds per image): [2.278, 3.301, 4.338, 5.266, 10.697] (4.734)



File: fulltestnewpromptshuffled2.txt
Times: [24.607833, 35.081481, 48.332297, 58.486441, 86.975065] (48.021)
Average Times (seconds per image): [0.652, 2.734, 4.356, 6.98, 42.217] (6.515)



File: fulltestshuffled.txt
Times: [4.511034, 36.680637, 45.971984, 53.154578, 96.566904] (43.415)

In [8]:
import re
import pandas as pd
import warnings

df = pd.DataFrame(columns=["Model", "Load Model Time", "Load Image Time", "Inference Time"])

#suppress all those pesky warnings
warnings.filterwarnings("ignore")


for key in content:
    output = content[key]
    # Convert list to string if necessary
    if isinstance(output, list):
        output = " ".join(output)

    #print(type(output), output)
    try:
        # Find "Time to load model: " and extract the float after it
        load_model_time = re.search(r"Time to load model: (\d+\.\d+)", output).group(1)
        # Find "Time to load image: " and extract the float after it
        load_image_time = re.search(r"Time to load image: (\d+\.\d+)", output).group(1)
        # Find "Time for inference: " and extract the float after it
        inference_time = re.search(r"Time for inference: (\d+\.\d+)", output).group(1)

        #Add to df
        df = df.append({"Model": key, "Load Model Time": load_model_time, "Load Image Time": load_image_time, "Inference Time": inference_time}, ignore_index=True)
    except:
        #No times to calculate
        pass

print(df)

     Model Load Model Time Load Image Time Inference Time
0   000001            7.65            0.00           9.73
1   000002            6.85            0.00          10.24
2   000003            7.39            0.00          10.60
3   000004            7.01            0.00           9.14
4   000005            6.91            0.00          10.14
..     ...             ...             ...            ...
95  000096            7.10            0.00           9.43
96  000097            6.93            0.00          11.09
97  000098            6.81            0.00           9.07
98  000099            6.90            0.00           9.48
99  000100            6.83            0.00          11.33

[100 rows x 4 columns]


In [17]:
#Grab some stats from df

average_load_model_time = df["Load Model Time"].astype(float).mean()
average_load_image_time = df["Load Image Time"].astype(float).mean()
average_inference_time = df["Inference Time"].astype(float).mean()

print(f"Average Load Model Time: {average_load_model_time:.2f}s")
print(f"Average Load Image Time: {average_load_image_time:.2f}s")
print(f"Average Inference Time: {average_inference_time:.2f}s")

total_time_avg = sum(times) / len(times)
print(f"Average Time for everything else: {(total_time_avg-(average_load_model_time + average_load_image_time + average_inference_time)):.2f}s")

print(f"Average Total Time: {total_time_avg:.2f}s")

Average Load Model Time: 6.97s
Average Load Image Time: 0.00s
Average Inference Time: 10.13s
Average Time for everything else: 5.22s
Average Total Time: 22.32s
